Python code to clean the data from the og file. Only for the rows: 19-24
Questions: County and Postal code - should I match with coordinates?
System and Model Number - match with 

In [ ]:
# Install pandas and openpyxl if needed
# !pip install pandas openpyxl


In [ ]:
import pandas as pd
import numpy as np

# Path to your Excel file
file_path = r"C:our-file-path.xlsx"

try:
    # Load the Excel file
    df = pd.read_excel(file_path)

  # Convert 'Driver Postal Code' and 'User ID' to numeric data types
    df['Driver Postal Code'] = pd.to_numeric(df['Driver Postal Code'], errors='coerce')  # Convert and set errors as NaN if conversion fails
    df['User ID'] = pd.to_numeric(df['User ID'], errors='coerce')

    # Print the data types of each column in the DataFrame after conversion
    print("Data types of the columns after conversion:")
    print(df.dtypes)
    print()  # Print a newline for better separation

    # Iterate through each column in the DataFrame and print unique values
    for column in df.columns:
        print(f"Unique values in '{column}':")
        unique_values = df[column].dropna().unique()  # Drop NA values to see only actual data
        print(unique_values)
        print()  # Print a newline for better separation of output for each column

except Exception as e:
    print("An error occurred:", e)

Cleaning County and Driver Postal Code. We don't have any specific data that can help navigate what Postal code it could be on empty rows we fill it in with 'Unknown'. For County wre can match the Driver Postal codes for eother users to identify the blank values. 

In [11]:
# Check unique values in the 'County' column to ensure correct spelling
print(df['County'].unique())

# make sure there are entries which match our conditions
print(df['County'].str.strip().str.lower().value_counts())

['Santa Clara County' nan 'San Mateo County']
County
santa clara county    147263
san mateo county       27487
Name: count, dtype: int64


In [ ]:
# Cleaning "Driver Postal Code": Replace NaN values with 0
df['Driver Postal Code'] = pd.to_numeric(df['Driver Postal Code'], errors='coerce')  # Convert to numeric, making non-convertible values NaN
df['Driver Postal Code'].fillna(0, inplace=True)  # Replace NaNs with 0

# Optionally, you can verify the conversion and filling:
print(df['Driver Postal Code'].head())  # Print the first few entries to check the changes

# 2. Extract unique postal codes where 'County' is 'Santa Clara County'
santa_clara_postal_codes = df[df['County'] == 'Santa Clara County']['Driver Postal Code'].unique()
print("Unique Driver Postal Codes for Santa Clara County:")
print(santa_clara_postal_codes)

# 3. Extract unique postal codes where 'County' is 'San Mateo County'
san_mateo_postal_codes = df[df['County'] == 'San Mateo County']['Driver Postal Code'].unique()
print("Unique Driver Postal Codes for San Mateo County:")
print(san_mateo_postal_codes)

In [13]:
# Create mapping dictionaries
santa_clara_dict = {code: 'Santa Clara County' for code in santa_clara_postal_codes}
san_mateo_dict = {code: 'San Mateo County' for code in san_mateo_postal_codes}

# Combine dictionaries
postal_code_to_county_dict = {**santa_clara_dict, **san_mateo_dict}

# Function to determine county based on postal code
def assign_county(row):
    if pd.isna(row['County']) and row['Driver Postal Code'] in postal_code_to_county_dict:
        return postal_code_to_county_dict[row['Driver Postal Code']]
    else:
        return row['County']

# Apply the function to the dataframe, only if County is NaN
df['County'] = df.apply(lambda row: assign_county(row) if pd.isna(row['County']) else row['County'], axis=1)

# Check to see how many still have NaN
print(f"Remaining NaN in County: {df['County'].isna().sum()}")

# Optionally fill any remaining NaN county values with 'Unknown'
df['County'] = df['County'].fillna('Unknown')

Remaining NaN in County: 402


In [ ]:
# Cleaning "Ended By": changing blanks (NaNs) to "Unknown"
df['Ended By'] = df['Ended By'].fillna('Unknown')

# Cleaning "User ID": Convert non-numeric 'User ID' to NaN, then replace NaNs and 0 with zero
df['User ID'] = pd.to_numeric(df['User ID'], errors='coerce')  # This conversion sets non-numeric to NaN
df['User ID'] = df['User ID'].fillna(0)  # Directly assigning back to the DataFrame column

# Count how many User IDs are now marked as '0'
unregistered_count = (df['User ID'] == 0).sum()
print(f"Count of unregistered users (marked as 0) in User ID: {unregistered_count}")

# Fill empty values in 'Model Number' with 'Unknown'
df['Model Number'] = df['Model Number'].fillna('Unknown')

# Handling columns with excessive missing data by either deleting them or filling in missing values NOT USING NOW
# columns_to_check = ['System S/N', 'Model Number']
#for column in columns_to_check:
    #missing_percentage = df[column].isna().sum() / len(df)
    #if missing_percentage > 0.30:
        # Remove column if more than 30% data is missing
        #df = df.drop(column, axis=1)  # Modify the DataFrame directly
       # print(f"Deleted column {column} due to {missing_percentage*100:.2f}% missing values.")
    #else:
        # Fill missing values with 0 directly in the DataFrame for numeric columns
       # df[column] = df[column].fillna(0)
       # print(f"Filled missing values in column {column} with 0.")

# Save the cleaned DataFrame back to an Excel file
output_path = r'C:our-file-path.xlsx'
df.to_excel(output_path, index=False)
print("Data cleaning completed and file saved.")

Count of unregistered users (marked as 0) in User ID: 8171
Deleted column System S/N due to 30.43% missing values.
Deleted column Model Number due to 30.43% missing values.
Data cleaning completed and file saved.
